In [ ]:
!git clone https://github.com/LeGiangK62/GNN-Resource-Management.git
%cd GNN-Resource-Management/NewDir

Cloning into 'GNN-Resource-Management'...
remote: Enumerating objects: 209, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 209 (delta 110), reused 158 (delta 62), pack-reused 0
Receiving objects: 100% (209/209), 236.41 KiB | 1.27 MiB/s, done.
Resolving deltas: 100% (110/110), done.
/content/GNN-Resource-Management/NewDir


In [ ]:
%%capture
import torch
!pip install torch_geometric

# Optional dependencies:
if torch.cuda.is_available():
  !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
else:
  !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
#

In [1]:
import torch
import numpy as np

from torch.nn import Sequential as Seq, Linear as Lin, ReLU, Sigmoid
from torch_geometric.data import HeteroData
from torch_geometric.loader import DataLoader
from torch_geometric.nn import Linear, HGTConv

from WSN_GNN import generate_channels_wsn

# Create HeteroData from the wireless system

In [192]:
#region Create HeteroData from the wireless system
def convert_to_hetero_data(channel_matrices):
    graph_list = []
    num_sam, num_aps, num_users = channel_matrices.shape
    for i in range(num_sam):
        x1 = torch.ones(num_users, 1)
        x2 = torch.ones(num_users, 1)  # power allocation
        x3 = torch.ones(num_users, 1)  # ap selection?
        user_feat = torch.cat((x1,x2,x3),1)  # features of user_node
        ap_feat = torch.zeros(num_aps, num_aps_features)  # features of user_node
        edge_feat_uplink = channel_matrices[i, :, :].reshape(1, -1)
        edge_feat_downlink = channel_matrices[i, :, :].reshape(1, -1)
        graph = HeteroData({
            'user': {'x': user_feat},
            'ap': {'x': ap_feat}
        })
        # Create edge types and building the graph connectivity:
        graph['user', 'uplink', 'ap'].edge_attr = torch.tensor(edge_feat_uplink, dtype=torch.float)
        graph['ap', 'downlink', 'user'].edge_attr = torch.tensor(edge_feat_downlink, dtype=torch.float)
        graph['user', 'uplink', 'ap'].edge_index = torch.tensor(adj_matrix(num_users, num_aps).transpose(), 
                                                                dtype=torch.int64)
        graph['ap', 'downlink', 'user'].edge_index = torch.tensor(adj_matrix(num_aps, num_users).transpose(),
                                                                dtype=torch.int64)

        # graph['ap', 'downlink', 'user'].edge_attr  = torch.tensor(edge_feat_downlink, dtype=torch.float)
        graph_list.append(graph)
    return graph_list


def adj_matrix(num_from, num_dest):
    adj = []
    for i in range(num_from):
        for j in range(num_dest):
            adj.append([i, j])
    return np.array(adj)


# Build Heterogeneous GNN

In [180]:
#region Build Heterogeneous GNN
class HetNetGNN(torch.nn.Module):
    def __init__(self, data, hidden_channels, out_channels, num_heads, num_layers):
        super().__init__()

        self.lin_dict = torch.nn.ModuleDict()
        for node_type in data.node_types:
            self.lin_dict[node_type] = Linear(-1, hidden_channels)

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HGTConv(hidden_channels, hidden_channels, data.metadata(),
                           num_heads, group='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

        self.lin1 = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        original = x_dict['user'].clone
        x_dict = {
            node_type: self.lin_dict[node_type](x).relu_()
            for node_type, x in x_dict.items()
        }

        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)

        original = x_dict['user'] # not original
        power = self.lin(x_dict['user'])
        ap_selection = self.lin1(x_dict['user'])
        ap_selection = torch.abs(ap_selection).int()
        
        out = torch.cat((original[:,1].unsqueeze(-1), power[:,1].unsqueeze(-1), ap_selection[:,1].unsqueeze(-1)), 1)
        return out

class EdgeConv(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def foward(self, graph, inputs):
        return 1


class RoundActivation(torch.nn.Module):
    def forward(self, x):
        return torch.round(torch.abs(x))

#endregion


## New GNN

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import MessagePassing
import torch_geometric.utils as pyg_utils
import torch.nn.functional as F

def MLP(channels, batch_norm=True):
    layers = []
    for i in range(1, len(channels)):
        layers.append(nn.Linear(channels[i - 1], channels[i]))
        layers.append(nn.ReLU())
        if batch_norm:
            layers.append(nn.BatchNorm1d(channels[i]))
    return nn.Sequential(*layers)

class EdgeConv(MessagePassing):
    def __init__(self, input_dim, node_dim, aggr='mean', **kwargs):
        super(EdgeConv, self).__init__(aggr=aggr)
        self.lin = MLP([input_dim, 32])
        self.res_lin = nn.Linear(node_dim, 32)

    def forward(self, x, edge_index, edge_attr):
        x = self.lin(edge_attr)  # Process edge attributes with MLP
        return self.propagate(edge_index, x=x)

    def message(self, x_j):
        return x_j

    def update(self, aggr_out, x):
        return aggr_out + self.res_lin(x)

class RGCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = EdgeConv(input_dim=4, node_dim=1)
        self.conv2 = EdgeConv(input_dim=66, node_dim=32)
        self.conv3 = EdgeConv(input_dim=66, node_dim=32)
        self.mlp = MLP([32, 16])
        self.mlp = nn.Sequential(self.mlp, nn.Linear(16, 1), nn.Sigmoid())

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        x = self.conv1(x, edge_index, edge_attr)
        x = self.conv2(x, edge_index, edge_attr)
        x = self.conv3(x, edge_index, edge_attr)
        x = self.mlp(x)

        return x

# Note: The input data format in `torch_geometric` might differ from what is used in the original code.
# Make sure to preprocess your input data accordingly to create `data` objects for the RGCN model.


# Training and Testing functions

In [194]:
#region Training and Testing functions
def loss_function(output, batch, is_train=True):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_user = batch['user']['x'].shape[0]
    num_ap = batch['ap']['x'].shape[0]
    ##
    channel_matrix = batch['user', 'ap']['edge_attr']
    ##
#     power_max = batch['user']['x'][:, 0]
#     power = batch['ue']['x'][:, 1]
#     ap_selection = batch['ue']['x'][:, 2]
    power_max = output[:, 0]
    power = output[:, 1]
    ap_selection = output[:, 2]
    ##
    ap_selection = ap_selection.int()
    index = torch.arange(num_user)

    G = torch.reshape(channel_matrix, (-1, num_ap, num_user))
    # P = torch.reshape(power, (-1, num_ap, num_user)) #* p_max
    P = torch.zeros_like(G, requires_grad=True).clone()
    P[0, ap_selection[index], index] = power_max * power
    ##
    
    # new_noise = torch.from_numpy(noise_matrix).to(device)
    desired_signal = torch.sum(torch.mul(P, G), dim=1).unsqueeze(-1)
    G_UE = torch.sum(G, dim=2).unsqueeze(-1)
    all_signal = torch.matmul(P.permute((0,2,1)), G_UE)
    interference = all_signal - desired_signal #+ new_noise
    rate = torch.log(1 + torch.div(desired_signal, interference))
    sum_rate = torch.mean(torch.sum(rate, 1))
    mean_power = torch.mean(torch.sum(P.permute((0,2,1)), 1))

    if is_train:
        return torch.neg(sum_rate / mean_power)
    else:
        return sum_rate / mean_power



def train(data_loader):
    model.train()
    device_type = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    total_examples = total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()
        batch = batch.to(device_type)
        # batch_size = batch['ue'].batch_size
        out = model(batch.x_dict, batch.edge_index_dict)
        tmp_loss = loss_function(out, batch, True)
        tmp_loss.backward()
        optimizer.step()
        #total_examples += batch_size
        total_loss += float(tmp_loss) #* batch_size

    return total_loss #/ total_examples


def test(data_loader):
    model.eval()
    device_type = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    total_examples = total_loss = 0
    for batch in data_loader:
        batch = batch.to(device_type)
        # batch_size = batch['ue'].batch_size
        out = model(batch.x_dict, batch.edge_index_dict)
        tmp_loss = loss_function(out, batch, False)
        #total_examples += batch_size
        total_loss += float(tmp_loss) #* batch_size

    return total_loss #/ total_examples
#endregion



# Main

In [5]:
K = 3  # number of APs
N = 5  # number of nodes
R = 10  # radius

num_users_features = 3
num_aps_features = 3

num_train = 2  # number of training samples
num_test = 4  # number of test samples

reg = 1e-2
pmax = 1
var_db = 10
var = 1 / 10 ** (var_db / 10)
var_noise = 10e-11

power_threshold = 2.0

X_train, noise_train, pos_train, adj_train, index_train = generate_channels_wsn(K, N, num_train, var_noise, R)
X_test, noise_test, pos_test, adj_test, index_test = generate_channels_wsn(K + 1, N + 10, num_test, var_noise, R)

In [28]:
# Maybe need normalization here
train_data = convert_to_hetero_data(X_train)
test_data = convert_to_hetero_data(X_test)

batchSize = 1

train_loader = DataLoader(train_data, batchSize, shuffle=True, num_workers=1)
test_loader = DataLoader(test_data, batchSize, shuffle=True, num_workers=1)

In [189]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data = train_data[0]
data = data.to(device)

model = HetNetGNN(data, hidden_channels=64, out_channels=4, num_heads=2, num_layers=1)
model = model.to(device)

# # print(data.edge_index_dict)
# with torch.no_grad():
#     output = model(data.x_dict, data.edge_index_dict)
# print(output)
# print(data)

# data = test_data[0]
# data = data.to(device)
#
# with torch.no_grad():
#     output = model(data.x_dict, data.edge_index_dict)
#     print(output)


## Training and testing

In [195]:

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.9)

for epoch in range(1, 101):
    loss = train(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Loss: {loss:.4f}, Test Reward: {test_acc:.4f}')

Epoch: 001, Train Loss: 47.0816, Test Reward: -29.8976
Epoch: 002, Train Loss: 29.0778, Test Reward: -20.5468
Epoch: 003, Train Loss: 20.1887, Test Reward: -15.2608
Epoch: 004, Train Loss: 15.2918, Test Reward: -11.8585
Epoch: 005, Train Loss: 11.9738, Test Reward: -9.5137
Epoch: 006, Train Loss: 9.6070, Test Reward: -7.8197
Epoch: 007, Train Loss: 7.9350, Test Reward: -6.5465
Epoch: 008, Train Loss: 6.7002, Test Reward: -5.5615
Epoch: 009, Train Loss: 5.7096, Test Reward: -4.7869
Epoch: 010, Train Loss: 4.9084, Test Reward: -4.1693
Epoch: 011, Train Loss: 4.3020, Test Reward: -3.6669
Epoch: 012, Train Loss: 3.7791, Test Reward: -3.2559
Epoch: 013, Train Loss: 3.3627, Test Reward: -2.9148
Epoch: 014, Train Loss: 3.0164, Test Reward: -2.6290
Epoch: 015, Train Loss: 2.7255, Test Reward: -2.3874
Epoch: 016, Train Loss: 2.4790, Test Reward: -2.1814
Epoch: 017, Train Loss: 2.2738, Test Reward: -2.0038
Epoch: 018, Train Loss: 2.0867, Test Reward: -1.8507
Epoch: 019, Train Loss: 1.9296, Test 

In [191]:
model = HetNetGNN(data, hidden_channels=64, out_channels=4, num_heads=2, num_layers=1)
model = model.to(device)

model.train()
device_type = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
total_examples = total_loss = 0
for batch in train_loader:
    optimizer.zero_grad()
    batch = batch.to(device_type)
    break
print(batch)

output = model(batch.x_dict, batch.edge_index_dict)
print(output)

HeteroDataBatch(
  user={
    x=[5, 3],
    batch=[5],
    ptr=[2]
  },
  ap={
    x=[3, 3],
    batch=[3],
    ptr=[2]
  },
  (user, uplink, ap)={
    edge_attr=[15, 1],
    edge_index=[2, 15]
  },
  (ap, downlink, user)={
    edge_attr=[15, 1],
    edge_index=[2, 15]
  }
)
tensor([[ 0.1879, -0.1242,  0.0000],
        [ 0.1879, -0.1242,  0.0000],
        [ 0.1879, -0.1242,  0.0000],
        [ 0.1879, -0.1242,  0.0000],
        [ 0.1879, -0.1242,  0.0000]], grad_fn=<CatBackward0>)


In [196]:
batch

HeteroDataBatch(
  user={
    x=[5, 3],
    batch=[5],
    ptr=[2]
  },
  ap={
    x=[3, 3],
    batch=[3],
    ptr=[2]
  },
  (user, uplink, ap)={
    edge_attr=[15, 1],
    edge_index=[2, 15]
  },
  (ap, downlink, user)={
    edge_attr=[15, 1],
    edge_index=[2, 15]
  }
)

In [197]:
batch.edge_index_dict

{('user',
  'uplink',
  'ap'): tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4],
         [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]]),
 ('ap',
  'downlink',
  'user'): tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
         [0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4]])}

In [211]:
for edge_type, edge_index in batch.edge_index_dict.items():
    x = edge_type
    src_type, _, dst_type = edge_type
    print(edge_type)
    edge_type = '__'.join(edge_type)
    print(src_type, dst_type)
    print(edge_type)
    

('user', 'uplink', 'ap')
user ap
user__uplink__ap
('ap', 'downlink', 'user')
ap user
ap__downlink__user


In [201]:
src_type,dst_type

('ap', 'user')

In [215]:
X = (edge_type.split('__'))

In [214]:
type(x)

tuple

In [216]:
type(X)

list

In [ ]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG
from torch_geometric.nn import SAGEConv, to_hetero


dataset = OGB_MAG(root='./data', preprocess='metapath2vec', transform=T.ToUndirected())
data = dataset[0]